In [1]:
import os
import time
import yaml
import re
import json
import requests
import pickle

from openai import OpenAI
import pandas as pd
import numpy as np
import ipywidgets as widgets # interactive display
from tqdm import tqdm # progress bar
from dotenv import load_dotenv
from collections import defaultdict

In [2]:
result = pd.read_pickle('result/제공_네이버쇼핑리뷰_S2_batch100_감성테마.pkl')

In [3]:
result

,raw_text,issue,sentiment,sentiment_all,theme
0,조아용 조아용 조아용 ㅎㅎ,좋은 제품,긍정,긍정,제품 품질 및 성능
1,너무 만족하며 사용중입니다,만족,긍정,긍정,고객 만족 및 추천
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 배송,긍정,긍정,배송 및 설치 서비스
3,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정,배송 및 설치 서비스
4,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정,고객 만족 및 추천
...,...,...,...,...,...
2468,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,야외 사용 기대,긍정,긍정,캠핑 및 야외 활용
2469,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,대대만족,긍정,긍정,제품 만족도
2470,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,아늑한 영화관 느낌,긍정,긍정,디자인 및 인테리어
2471,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,이동 편리,긍정,긍정,사용 편의성


In [9]:
# 'issue'별 'sentiment' 개수 계산 및 'total' 컬럼 추가 후 내림차순 정렬
issue_sentiment_counts = result.groupby(['issue', 'theme', 'sentiment']).size().unstack(fill_value=0)
issue_sentiment_counts['total'] = issue_sentiment_counts.sum(axis=1)
issue_sentiment_counts = issue_sentiment_counts.sort_values(by='total', ascending=False)
issue_sentiment_counts

,sentiment,긍정,부정,중립,total
issue,theme,,,,
빠른 배송,배송 및 설치 서비스,180,0,1,181
좋은 제품,제품 품질 및 성능,99,0,0,99
무거움,기타,0,69,10,79
저렴한 가격,가격 및 가치,50,0,0,50
만족,고객 만족 및 추천,46,0,0,46
...,...,...,...,...,...
빠른 속도,기타,1,0,0,1
빠른 응답속도,사용 편의성,1,0,0,1
빠른 주문,설치 및 서비스,0,0,1,1


In [11]:
# Check if 'issue' and 'theme' have a 1:1 correspondence
issue_theme_unique_pairs = result[['issue', 'theme']].drop_duplicates()
issue_theme_duplicated = issue_theme_unique_pairs.duplicated(subset='issue', keep=False)

# If there are any duplicated 'issue', it means 'issue' and 'theme' do not have a 1:1 correspondence
if issue_theme_duplicated.any():
    print("issue와 theme는 1:1 대응이 되지 않습니다.")
else:
    print("issue와 theme는 1:1 대응이 됩니다.")

issue와 theme는 1:1 대응이 됩니다.


In [10]:
# 엑셀 파일로 저장
issue_sentiment_counts.to_excel('result/제공_네이버쇼핑리뷰_S3_batch100_기회영역값.xlsx')